In [3]:
from dotenv import load_dotenv
load_dotenv( override=True)

True

In [ ]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Answer all the questions asked by the user diligently.
If you don't know the answer, just say that you don't know.
"""

openai_client = OpenAI()
nest_asyncio.apply()

@traceable
def generate_response(question: str):
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    response = generate_response(question)
    return response.choices[0].message.content

#  A simple chatbot without chat history
@traceable
def chat_bot():
    count=1
    while True:
        question=input("Enter your question: ")
        if question == "exit":
            break
        response = langsmith_rag(question, langsmith_extra={"metadata": {"Question Number": str(count)}})
        count=count+1;
        print(response)



chat_bot()

Humans created AI through a combination of computer science, mathematics, and cognitive science. The development of AI can be traced back to the mid-20th century, with key milestones including:

1. **Theoretical Foundations**: Early work in logic, algorithms, and computation laid the groundwork for AI. Alan Turing's concept of a "universal machine" and his work on the Turing Test were pivotal.

2. **Symbolic AI**: In the 1950s and 1960s, researchers developed systems that used symbolic reasoning to solve problems. This approach focused on manipulating symbols to represent knowledge.

3. **Machine Learning**: In the 1980s and 1990s, the focus shifted to machine learning, where algorithms learn from data rather than relying solely on pre-programmed rules. Techniques like neural networks gained popularity.

4. **Big Data and Computing Power**: The explosion of data and advancements in computing power in the 21st century enabled more complex models and algorithms, leading to breakthroughs 